In [20]:
import numpy as np
from typing import  List, Tuple, Dict, Callable,Any

def Cost_function(r:np.ndarray,args:Any)->np.ndarray:
    # row_reduction = args['row_reduction'] 
 
    # r[:,:,0]+= (np.floor(r[:,:,3:])*(((row_reduction.reshape(1,1,row_reduction.size)).repeat(r.shape[0],axis=0)).repeat(r.shape[1],axis=1))).sum(axis=2)
    return r
def Res_function(R:np.ndarray,args:Any)->np.ndarray:
    func = args['func']
    R_:np.ndarray = np.zeros_like(R)
    for i,f in enumerate(func):
        if f is not None:
            R_[:,:,i] = f(R[:,:,i])
    return R_

class ESP:
    def __init__(self,
                 r_max:np.ndarray,
                 r:np.ndarray)->None:
                #  type_r:np.ndarray,
                #  cost_function:Callable,
                #  Res_function:Callable,
                #  args:Any) -> None:
        __slots__ = 'MAX','c','r','n','n_res','r_max','U','R','N','O','I','V','path','current','best_path','best_cost'
        self.MAX :int = 1000_000
        self.r:np.ndarray = r 
        self.n: int = r.shape[0]
        self.n_res:int  = r.shape[-1]
        self.r_max:np.ndarray = r_max
        self.I:np.ndarray = np.zeros(self.MAX,dtype=int)
        self.U:np.ndarray = np.ones((self.MAX,self.n),dtype=bool)
        self.R:np.ndarray = np.zeros((self.MAX,self.n_res),dtype=float) #0:cost , 1: r[c,t,q]
        self.O:np.ndarray = np.ones(self.MAX,dtype=bool)# True: not reached half-point or child-up
        self.N:np.ndarray = np.ones(self.MAX,dtype=bool)# True: has not assigned a label
        self.V:np.ndarray = np.zeros(self.MAX,dtype=int) #[vertex]
        self.P:np.ndarray = np.zeros((self.MAX,n),dtype=int)
        self.LB: np.ndarray = np.zeros(self.MAX,dtype=float)
        self.UB:float = 1000
        self.visit:np.ndarray = np.zeros((self.MAX,self.n),dtype=int)
        self.U[:,0] = False #False: not reachable, True: reachable
        self.N[0] = False
        self.visit[0,0] = 1
        self.min_values:np.ndarray = np.min(self.r[:,:,0],axis=1)
        self.current:np.ndarray =  np.where(~self.N)[0]
        self.best_path:List[List[int]] = []
        self.best_cost:List[float] = []
        
    def generate(self)->None:
        p:np.ndarray = np.where(self.O&~self.N)[0]#[:int(1000/self.n)]
        n_child:np.ndarray =self.U[p,:].sum(axis=1).astype(int) #get of number of children
        num_child:int = int(n_child.sum())
        new_child_index:np.ndarray = self.N.nonzero()[0][:num_child]
        vertex_child:np.ndarray= np.where(self.U[p,:])[1] # the corresponding vertex
        vertex_parent:np.ndarray= np.repeat(self.V[p],n_child).astype(int)
        parent_index:np.ndarray = np.repeat(p,n_child).astype(int)
        
        self.O[p] = False #set parents closed since they all childed up
        self.N[new_child_index] = False #set new children as not new anymore
        self.V[new_child_index] = vertex_child # set the vertex of new children

        self.P[new_child_index,:] = self.P[parent_index,:] # set the path of new children
        # print(self.I[parent_index]+1)
        self.P[new_child_index,self.I[parent_index]+1] = vertex_child # set the path of new children
        self.I[new_child_index] = self.I[parent_index]+1

        self.U[new_child_index,vertex_child] = False #make children unreachable
        self.R[new_child_index,:] = self.r[vertex_parent,vertex_child,:]+self.R[parent_index,:]

        self.LB[new_child_index] = self.R[new_child_index,0]+(self.min_values*self.U[new_child_index,:]).sum(axis=1)
        # self.R[new_child_index,0] = self.R[new_child_index,1]#+(np.floor(self.pi.reshape(1,len(self.pi))*self.R[new_child_index,2].reshape(num_child,1))*self.xi.reshape(1,len(self.pi))).sum(axis=1)
        self.U[new_child_index,:] &=self.U[parent_index,:]
        # self.U[new_child_index,:] &=(self.Res_function(np.repeat(self.R[new_child_index,np.newaxis , 1:],repeats=self.n,axis=1)+self.r[vertex_child,:,:],self.args)<=self.r_max).all(axis=2)
        self.U[new_child_index,:] &= ((self.R[new_child_index,np.newaxis,1:]+self.r[vertex_child,:,1:])<=self.r_max).all(axis=2)
        self.O[new_child_index[np.any(self.R[new_child_index,1:]>=self.r_max/2,axis=1)]] = False
        
        self.visit[new_child_index,:] += self.visit[parent_index,:]
        self.visit[new_child_index,vertex_child] = 1
        
    def sort_labels(self)->None:
        self.current:np.ndarray = np.where(~self.N)[0]
        idx:np.ndarray = self.current[np.lexsort([self.R[self.current,1],
                                                  (1-self.U[self.current,:]).sum(axis=1),
                                                  self.R[self.current,0],self.V[self.current]])]
        i:int = idx.size
        self.LB[:i] = self.LB[idx]
        self.LB[i:] = 0
        self.R[:i,:]= self.R[idx,:]
        self.R[i:,:] = 0
        self.N[:i] = False
        self.N[i:] = True
        self.V[:i]= self.V[idx]
        self.V[i:] = 0
        self.U[:i,:] = self.U[idx,:]
        self.U[i:,:] = True
        self.O[:i]= self.O[idx]
        self.O[i:] = True
        self.P[:i,:] = self.P[idx,:]
        self.P[i:,:] = 0
        self.visit[:i,:] = self.visit[idx,:]
        self.visit[i:,:] = 0
        self.I[:i] = self.I[idx]
        self.I[i:] = 0
        self.current = np.arange(i)
    def domination(self)->None:
        # max_i = max([len(self.current[np.where(self.V[self.current]==i)[0]]) for i in range(1,n)])
        
        for i in range(self.n):
            idx = self.current[np.where(self.V[self.current]==i)[0]]
            r= np.hstack((self.R[idx,:],1-self.U[idx,:]))
            i_ = np.unique((idx[np.where(np.all(r[:, np.newaxis, :] >= r[np.newaxis, :, :],axis=2).astype(int)-np.eye(r.shape[0])==1)[0]])).astype(int)
            self.N[i_] = True
            self.V[i_] = 0
            self.R[i_,:] = 0
            self.U[i_,:] = True
            self.O[i_] = True
            self.P[i_,:] = 0
            self.visit[i_,:] = 0
            self.I[i_] = 0
            self.LB[i_] = 0
        self.sort_labels()
    
        
    def solve(self)->None:
        # self.sort_labels()
        # print(self.O[~self.N])
        while np.any(self.O&~self.N):
            # print(self.O[~self.N])
            self.generate()
            self.domination()
            self.sort_labels()
        self.concat()
        self.best_path = [[int(j) for j in p] for p in self.best_path]
        self.best_cost = [float(c) for c in self.best_cost]
    
    def concat(self)->None:
        for i in range(1,self.n):
            J:np.ndarray = np.where(self.V==i)[0]
            R:np.ndarray = self.R[J,:]
            V:np.ndarray = self.visit[J,:]
            
            
            r_comp:np.ndarray = R[:, np.newaxis, :] + R[np.newaxis, :, :]
            res_V:np.ndarray = V[:, np.newaxis, :] + V[np.newaxis, :, :]
            res_V[:,:,0]-=1
            res_V[:,:,i]-=1
            
            f,b = np.where(np.all(r_comp[:,:,1:]<=self.r_max,axis=2)*np.all( res_V<=1,axis=2))
            
            mask = f < b
            f, b = f[mask], b[mask]
            if len(f) > 0:
                r_comp_min = r_comp[f, b, 0]
                i_s = np.argmin(r_comp_min)
                fs, bs = J[f[i_s]], J[b[i_s]]
                path_f, path_b = self.P[fs][:self.I[fs]+1].tolist(), self.P[bs][:self.I[bs]+1].tolist()
                path_b.reverse()
                self.best_path.append(path_f + path_b[1:])
                self.best_cost.append(r_comp_min[i_s])
                if r_comp_min[i_s] < self.UB:
                    self.UB = r_comp_min[i_s]

eps = ESP(r_max,r)

# eps.solve()

In [29]:
%timeit ESP(r_max,r)

12.9 ms ± 6.14 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
%%cython -a 

from libcpp.vector cimport vector
# cdef int num_resources = r_max.size() + 1

cdef void build_new_path_resources_reachables(
    int parent_node, int parent_index, int child_node, int direction,
    double[:,:,:] graph_data, vector[vector[vector[int]]]& L_p,
    vector[vector[vector[double]]]& L_r, vector[vector[vector[int]]]& L_reach,
    vector[int]& child_path, vector[double]& child_resource, vector[int]& child_reachables,
    int& child_halfpoint, int num_resources):

    child_path = L_p[parent_node][parent_index][:]
    child_path.push_back(child_node)

    child_resource = L_r[parent_node][parent_index][:]
    child_reachables = L_reach[parent_node][parent_index][:]
    child_reachables[child_node] = 0
    child_halfpoint = 0

    for j in range(num_resources):
        if direction == 1:
            child_resource[j] += graph_data[parent_node, child_node, j]
        else:
            child_resource[j] -= graph_data[child_node, parent_node, j]

cdef void update_reachables(
    int child_node, int direction, double[:,:,:] graph_data,
    vector[int]& child_reachables, vector[double]& child_resource, int num_resources):

    for i in adj_list[child_node]:
        if child_reachables[i] == 1:
            for j in range(1, num_resources):
                if (direction == 1 and child_resource[j] + graph_data[child_node, i, j] > r_max[j-1]) or (direction == 0 and child_resource[j] - graph_data[i, child_node, j] < 0):
                    child_reachables[i] = 0
                    break

cdef int check_halfpoint(
    int direction, vector[double]& child_resource, int num_resources):

    for j in range(1, num_resources):
        if (direction == 1 and child_resource[j] >= r_max[j-1] / 2) or (direction == 0 and child_resource[j] <= r_max[j-1] / 2):
            return 1
    return 0

cdef void check_dominance(
    int child_node, vector[double]& child_resource, vector[int]& child_reachables,
    vector[vector[vector[int]]]& L_p, vector[vector[vector[double]]]& L_r,
    vector[vector[vector[int]]]& L_reach):

    k = 0
    while k < L_p[child_node].size():
        ind = dominanceCheck(child_resource, L_r[child_node][k], child_reachables, L_reach[child_node][k])
        if ind == 1:
            break
        elif ind == 2:
            L_p[child_node].erase(L_p[child_node].begin() + k)
            L_r[child_node].erase(L_r[child_node].begin() + k)
            L_reach[child_node].erase(L_reach[child_node].begin() + k)
        else:
            k += 1

cdef void process_node(
    int parent_node, int parent_index, int direction, double[:,:,:] graph_data,
    vector[vector[vector[int]]]& L_p, vector[vector[vector[double]]]& L_r,
    vector[vector[vector[int]]]& L_reach, vector[vector[int]]& adj_list):

    cdef int i, child_node
    cdef vector[int] child_path 
    cdef vector[double] child_resource 
    cdef vector[int] child_reachables 
    cdef int child_halfpoint

    for i in range(adj_list[parent_node].size()):
        if L_reach[parent_node][parent_index][adj_list[parent_node][i]] == 1:
            child_node = adj_list[parent_node][i]
            print(f"Processing child node: {child_node}")

            build_new_path_resources_reachables(
                parent_node, parent_index, child_node, direction, graph_data,
                L_p, L_r, L_reach, child_path, child_resource, child_reachables,
                child_halfpoint, num_resources)

            update_reachables(child_node, direction, graph_data, child_reachables, child_resource, num_resources)

            child_halfpoint = check_halfpoint(direction, child_resource, num_resources)

            check_dominance(child_node, child_resource, child_reachables, L_p, L_r, L_reach)


Error compiling Cython file:
------------------------------------------------------------
...

cdef void update_reachables(
    int child_node, int direction, double[:,:,:] graph_data,
    vector[int]& child_reachables, vector[double]& child_resource, int num_resources):

    for i in adj_list[child_node]:
             ^
------------------------------------------------------------

C:\Users\rezam\.ipython\cython\_cython_magic_aa73adcdabe613989187731a149486f18e318f7d.pyx:30:13: undeclared name not builtin: adj_list
performance hint: C:\Users\rezam\.ipython\cython\_cython_magic_aa73adcdabe613989187731a149486f18e318f7d.pyx:33:82: Index should be typed for more efficient access
performance hint: C:\Users\rezam\.ipython\cython\_cython_magic_aa73adcdabe613989187731a149486f18e318f7d.pyx:33:85: Index should be typed for more efficient access

Error compiling Cython file:
------------------------------------------------------------
...
    vector[int]& child_reachables, vector[double]& child_r

In [65]:
from scipy.optimize import linprog
def kp(r_max,min_res)->np.ndarray:
    res = linprog(min_res[:,0].T, A_ub= min_res[:,1:].T, b_ub=r_max, bounds=(0,1), method='highs')
    return res.fun
    

In [ ]:
U = eps.U[~eps.N,:]
U = U[1,:]
def prep_kp(r,U):
    r_ = r[:,U,:][U,:,:]
    r_ = r_.min(axis=0).T
    r_ = r_.reshape(1,r_.shape[0],r_.shape[1])
    return r_

In [61]:

r.astype(np.float16)

array([[[ 1.520e+00,  4.000e+00,  1.000e+00],
        [ 1.050e-01,  4.000e+00,  1.250e+00],
        [ 8.652e-01,  3.500e+00,  1.000e+00],
        [-2.320e+00,  6.000e+00,  1.500e+00],
        [ 1.225e+00,  5.500e+00,  1.250e+00],
        [ 3.545e+00,  9.500e+00,  1.000e+00],
        [-4.700e-01,  5.000e+00,  1.250e+00],
        [ 1.805e+00,  4.500e+00,  1.500e+00],
        [-1.200e-01,  1.500e+00,  1.500e+00],
        [-1.860e+00,  7.000e+00,  1.000e+00]],

       [[ 1.050e-01,  4.000e+00,  1.250e+00],
        [-2.300e-01,  4.000e+00,  1.500e+00],
        [ 3.410e+00,  9.000e+00,  1.250e+00],
        [-4.363e+00,  6.000e+00,  1.750e+00],
        [-2.500e-01,  5.000e+00,  1.500e+00],
        [-1.851e-01,  6.000e+00,  1.250e+00],
        [-1.620e+00,  5.500e+00,  1.500e+00],
        [-2.900e-01,  5.500e+00,  1.750e+00],
        [ 3.750e-01,  1.000e+00,  1.750e+00],
        [-2.730e+00,  2.000e+00,  1.250e+00]],

       [[ 8.652e-01,  3.500e+00,  1.000e+00],
        [ 3.410e+00,  9.000e+0

In [13]:
n = 10
q = np.random.randint(1,5,(1,n)).repeat(n,axis=0)*0.5
q +=q.T
q *=0.5
t = np.random.randint(1,11,(n,n))
t = (t+t.T)*0.5
c = np.round(np.random.random((n,n))*10-5,2)
c = c+c.T
c *=0.5
r = np.zeros((n,n,3))
r[:,:,0] = c
r[:,:,1] = t
r[:,:,2] = q
r_max = np.array([26,8]).astype(np.float64)


In [10]:
import numpy as np
def create():
    I = np.random.randint(1,10,(100000,10,10),dtype=np.int8)
    return I
I = np.random.randint(1,10,(100000,10,10),dtype=np.int8)
def create2(I):
    I.fill(0)
    return I

In [11]:
%timeit create()

219 ms ± 50 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%timeit create2(I)

671 μs ± 36 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [2]:
%load_ext cython

In [26]:
%%cython -a 

import numpy as np
cimport numpy as np


cpdef tuple intialize(np.ndarray[np.float64_t , ndim=3] r, np.ndarray[np.float64_t , ndim=1] r_max):
    # cdef np.ndarray[np.int8_t, ndim=3] I = np.zeros((100000, 10, 10), dtype=np.int8)
    cdef int n = r.shape[0]
    cdef int n_res = r.shape[2]
    cdef long long MAX = 1000000
    # print(1)
    cdef np.ndarray[np.uint8_t, ndim=1] I = np.zeros(MAX, dtype=np.uint8)
    # print(2)
    cdef np.ndarray[np.uint8_t, ndim=2, cast=True] U = np.ones((MAX, n), dtype=np.bool)
    # print(3)
    cdef np.ndarray[np.float64_t , ndim=2] R = np.zeros((MAX, n_res), dtype=np.float64)
    # print(4)
    cdef np.ndarray[np.uint8_t, ndim=1, cast=True] O = np.ones(MAX, dtype=np.bool)
    # print(5)
    cdef np.ndarray[np.uint8_t, ndim=1, cast=True] N = np.ones(MAX, dtype=np.bool)
    # print(6)
    cdef np.ndarray[np.uint8_t, ndim=1, cast=True] V = np.zeros(MAX, dtype=np.uint8)
    # print(7)
    cdef np.ndarray[np.uint8_t, ndim=2] P = np.zeros((MAX, n), dtype=np.uint8)
    # print(8)
    cdef np.ndarray[np.float64_t, ndim=1] lb = np.zeros(MAX, dtype=np.float64)
    # print(9)
    cdef double ub = 1000.0
    cdef np.ndarray[np.uint8_t, ndim=2, cast=True] visit = np.zeros((MAX, n), dtype=np.bool)
    # print(10)
    U[:, 0] = False
    N[0] = False
    visit[0, 0] = 1

    cdef np.ndarray[np.float64_t, ndim=1] min_values = np.min(r[:, :, 0], axis=1)
    cdef np.ndarray[np.float64_t, ndim=2] min_weights = np.min(r[:, :, 1:], axis=2)

    cdef np.ndarray[np.int64_t, ndim=1] current = np.where(~N)[0]
    return I,U,R,O,N,V,P,lb,ub,visit,min_values,current,min_weights

Content of stdout:
_cython_magic_30f7c1bc0554ef17f015d24e41fc90802801070b.c
c:\Users\rezam\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\rezam\.ipython\cython\_cython_magic_30f7c1bc0554ef17f015d24e41fc90802801070b.c(4960): warning C4244: '=': conversion from 'npy_intp' to 'int', possible loss of data
C:\Users\rezam\.ipython\cython\_cython_magic_30f7c1bc0554ef17f015d24e41fc90802801070b.c(4969): warning C4244: '=': conversion from 'npy_intp' to 'int', possible loss of data
   Creating library C:\Users\rezam\.ipython\cython\Users\rezam\.ipython\cython\_cython_magic_30f7c1bc0554ef17f015d24e41fc90802801070b.cp312-win_amd64.lib and object C:\Users\rezam\.ipython\cython\Users\rezam\.ipython\cython\_cython_magic_30f7c1bc0554ef17f015d24e41fc90802801070b.cp312-win_amd64.exp
Generating code
Finished generating code

In [27]:
I,U,R,O,N,V,P,lb,ub,visit,min_values,current,min_weights= intialize(r,r_max)

In [39]:
%%cython -a
# Import necessary modules
import numpy as np
cimport numpy as np
from libc.stdlib cimport malloc, free

def c_array_to_numpy(int n):
    # Allocate memory for the C array
    cdef int* c_array = <int*> malloc(n * sizeof(int))

    # Fill the C array with values
    for i in range(n):
        c_array[i] = i + 1
    print(1)
    # Convert the C array to a NumPy array
    cdef np.ndarray[np.int32_t, ndim=1] np_array = np.frombuffer(<char*>c_array, dtype=np.int32, count=n)
    print(2)
    # Optionally, you can return the NumPy array
    print(np_array)

    # Free the C array memory
    free(c_array)
    return np_array

Content of stdout:
_cython_magic_cfab9e24b76df64d4cf103dbe6763c2109393a66.c
c:\Users\rezam\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\rezam\.ipython\cython\_cython_magic_cfab9e24b76df64d4cf103dbe6763c2109393a66.c(6966): warning C4244: '=': conversion from 'Py_ssize_t' to 'long', possible loss of data
   Creating library C:\Users\rezam\.ipython\cython\Users\rezam\.ipython\cython\_cython_magic_cfab9e24b76df64d4cf103dbe6763c2109393a66.cp312-win_amd64.lib and object C:\Users\rezam\.ipython\cython\Users\rezam\.ipython\cython\_cython_magic_cfab9e24b76df64d4cf103dbe6763c2109393a66.cp312-win_amd64.exp
Generating code
Finished generating code

In [40]:
c_array_to_numpy(10)

1


ValueError: buffer is smaller than requested size